In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [4]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(r"D:\GS_WebScarping\project_11_eca\chromedriver.exe")
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    
    
    # num of cars
    try:
        num_cars_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="div-block-74"]')))
        if num_cars_card:
            num_cars = WebDriverWait(num_cars_card, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="text-block-16 ng-binding"]')))
            nums = num_cars.text.split(": ")
            cars = int(nums[1])
            print(f"{cars} number of cars found")
        else:
            print("No cars")
    except Exception as e:
        print("No cars main page")
    
    
    # get the car numbers per page
    try:
        main_card =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="container-results w-container"]'))) 
        if main_card:
            cards = main_card.find_elements(By.XPATH, './/div[@class="div-results-main ng-scope"]')
            if cards:
                cars_per_page = int(len(cards))
                print(f"{cars_per_page} number of cars found on a page")
            else:
                print("No cars found on the page")
    except Exception as e:
        print("No cars found on the page")
        
    
    
    car_count = 0
    results = []
    
    while car_count<cars:
        
        try:
            main_car_links= WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="container-results w-container"]')))
        
            for i in range(cars_per_page): #cars
                if car_count >= cars:
                    break
                    
                try:
                    main_car_links= WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="container-results w-container"]')))
                    if main_car_links:
                        car_links = WebDriverWait(main_car_links, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[text()="More Info"]')))
                        driver.execute_script("arguments[0].scrollIntoView();", car_links[i])
                    
                        import time
                        time.sleep(1) 
                        car_links[i].click()
                        
                        # dict
                        details= {}
                        # get the name
                        try:
                            top_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="container-21 w-container"]')))
                            if top_card:
                                # car name
                                car_title = WebDriverWait(top_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="text-block-results-car-make-model ng-binding"]')))
                                if car_title:
                                    details['Title'] = car_title.text.strip()
                                else:
                                    details['Title'] = "na"
                                    
                                # car_loc
                                car_dtime =WebDriverWait(top_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="text-block-sale-details ng-binding"]'))) 
                                if car_dtime:
                                    details['Date_time'] = car_dtime.text.strip()
                                else:
                                    details['Date_time'] = "na"
                                    
                        except Exception as e:
                            print("No car title")
                            
                        
                        # car img 
                        try:
                            car_img =WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, './/a[@id="w-lightbox"]')))  
                            if car_img:
                                img_src = car_img.find_element(By.TAG_NAME, "img")
                                if img_src:
                                    details['Images'] = img_src.get_attribute("src")
                                else:
                                    details['Images'] = "na"
                        except Exception as e:
                            print("no images")                    
                            
                        # details
                        try:
                            main_det_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="div-block-detail"]')))
                            if main_det_card:
                                # top_dets
                                top_dets =WebDriverWait(main_det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="div-block-dp-info-txt-x2"]'))) 
                                if top_dets:
                                    # blocks of information
                                    top_blocks = WebDriverWait(main_det_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="div-details-page-info-text"]')))
                                    for block in top_blocks:
                                        det_label = block.find_element(By.XPATH, './/h5[@class="heading-6"]').text.strip()
                                        det_val = block.find_element(By.XPATH, './/div[@class="ng-binding"]').text.strip()
                                        if det_label and det_val:
                                            details[det_label] = det_val
                                        else:
                                            print("No car details")
                                else:
                                    print("no top details")

                                # bottom info
                                bottom_info = WebDriverWait(main_det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="div-block-online-inspectiontxt"]')))
                                if bottom_info:
                                    bottom_infos = bottom_info.find_elements(By.XPATH, './/div[@class="ng-binding"]')
                                    if bottom_infos:
                                        hist = [info.text.strip() for info in bottom_infos]
                                        details['History'] = hist
                                    else:
                                        print("No car bottom info")
                                else:
                                    print("No car bottom info")
                        except Exception as e:
                            print("No car details")
                                        

                        # inspection
                        try:
                            insp_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="container-inspection-dp w-container"]'))) 
                            if insp_card:
                                insp = insp_card.find_element(By.XPATH, './/a[text()="View Inspection Report"]')
                                if insp:
                                    details['Inspec_report'] = insp.get_attribute("href")
                                else:
                                    details['Inspec_report']= "na"
                        except Exception as e:
                            print("No inspection report")
                                
                        results.append(details)
                        driver.back()
                        
                        car_count+=1
                except Exception as e:
                    print("No car links")
                        
            if car_count % 25 == 0: # --> 25%25 == 0 and 50%25== 0 --> issue here
                try:
                    next_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="page-numbers"])[2]')))
                    if next_card:
                        # find the leading links
                        next_link = next_card.find_elements(By.XPATH, './/a[@class="text-block-17 page-number ng-scope"]')
                        if next_link:
                            driver.execute_script("arguments[0].scrollIntoView();", next_link)
                            time.sleep(2)
                            next_link.click()
                            WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class = 'mb-3 card']")))
                            print("Hit the next button")
                except Exception as e:
                    print("No next found")
                    break    
                    
        except Exception as e:
            print("No car links")
            break
        
                
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("eca.csv", index=False)
    driver.quit()
    
path = "https://www.ebca.co.uk/stock-list?saleid=569"
scrape(path)

No cars main page
No cars found on the page


UnboundLocalError: cannot access local variable 'cars' where it is not associated with a value

### next button issue

In [3]:
df=pd.read_csv("eca.csv")
df

,Title,Date_time,Images,Lot No:,Reg:,Reg date:,Make:,Model:,Derivative:,Doors:,...,Transmission:,Fuel Type:,Mileage:,Former Keepers:,Inspec_report,Body Style:,MOT Expiry:,V5 Location:,Vendor:,History
0,2018 TOYOTA HILUX INVINCIBLE D-4D 4WD - 2393cc,11:00 Wednesday 5th February 2025,https://www.ebca.co.uk/images/imagecomingsoon.jpg,0,AO68HHT,12 October 2018,TOYOTA,HILUX,INVINCIBLE D-4D 4WD - 2393cc,0,...,Automatic,Diesel,151000 Correct,0,https://ebca-media.kfsnet.co.uk/i_folder/AO68H...,NaN,NaN,NaN,NaN,NaN
1,2014 TOYOTA AURIS EXCEL VVT-I CVT - 1797cc,11:00 Wednesday 5th February 2025,https://www.ebca.co.uk/images/imagecomingsoon.jpg,0,AU14RCZ,01 March 2014,TOYOTA,AURIS,EXCEL VVT-I CVT - 1797cc,5,...,Automatic,Hybrid,81000 Correct,0,https://ebca-media.kfsnet.co.uk/i_folder/AU14R...,Estate,NaN,NaN,NaN,NaN
2,2019 DS DS3 C-BACK PFRMC LN PTCH - 1200cc,11:00 Wednesday 5th February 2025,https://www.ebca.co.uk/images/imagecomingsoon.jpg,0,KM69VTW,30 November 2019,DS,DS3,C-BACK PFRMC LN PTCH - 1200cc,5,...,Manual,Petrol,30000 Correct,1,https://ebca-media.kfsnet.co.uk/i_folder/KM69V...,MPV,NaN,NaN,NaN,NaN
3,2016 LAND ROVER R ROVER EVOQUE SE TECH TD - 19...,11:00 Wednesday 5th February 2025,https://www.ebca.co.uk/images/imagecomingsoon.jpg,0,VK16NLY,31 March 2016,LAND ROVER,R,ROVER EVOQUE SE TECH TD - 1999cc,5,...,Automatic,Diesel,92000 Correct,2,https://ebca-media.kfsnet.co.uk/i_folder/VK16N...,Estate,NaN,NaN,NaN,NaN
4,2020 VAUXHALL CORSA SRI PREMIUM TURBO - 1199cc,11:00 Wednesday 5th February 2025,https://www.ebca.co.uk/images/imagecomingsoon.jpg,0,KE69ZGF,28 February 2020,VAUXHALL,CORSA,SRI PREMIUM TURBO - 1199cc,5,...,Manual,Petrol,40000 Correct,1,https://ebca-media.kfsnet.co.uk/i_folder/KE69Z...,Hatchback,NaN,NaN,NaN,NaN
5,2014 TOYOTA YARIS ICON + VVT-I - 1329cc,11:00 Wednesday 5th February 2025,https://ebca-media.azureedge.net/i_folder/GU14...,0,GU14SVN,25 June 2014,TOYOTA,YARIS,ICON + VVT-I - 1329cc,5,...,Manual,Petrol,40135 Correct,1,https://ebca-media.kfsnet.co.uk/i_folder/GU14S...,Hatchback,November 2025,Here,Yeomans Group,['Last Service - 07.11.2023 at 33670 miles (7 ...
6,2017 FORD FIESTA ZETEC TURBO - 998cc,11:00 Wednesday 5th February 2025,https://www.ebca.co.uk/images/imagecomingsoon.jpg,0,LF67FNK,23 September 2017,FORD,FIESTA,ZETEC TURBO - 998cc,5,...,Manual,Petrol,76000 Correct,1,https://ebca-media.kfsnet.co.uk/i_folder/LF67F...,Hatchback,NaN,NaN,NaN,NaN
7,2023 FORD PUMA ST-LINE MHEV - 999cc,11:00 Wednesday 5th February 2025,https://www.ebca.co.uk/images/imagecomingsoon.jpg,0,GV72OOH,14 January 2023,FORD,PUMA,ST-LINE MHEV - 999cc,5,...,Manual,Petrol,10000 Correct,0,https://ebca-media.kfsnet.co.uk/i_folder/GV72O...,Hatchback,NaN,NaN,NaN,NaN
8,2018 ALFA ROMEO GIULIA SPECIALE TD AUTO - 2143cc,11:00 Wednesday 5th February 2025,https://ebca-media.azureedge.net/i_folder/PY67...,0,PY67LNX,22 February 2018,ALFA ROMEO,GIULIA,SPECIALE TD AUTO - 2143cc,4,...,Automatic,Diesel,70373 Correct,2,https://ebca-media.kfsnet.co.uk/i_folder/PY67L...,Saloon,NaN,NaN,NaN,NaN
9,2021 NISSAN QASHQAI N-MOTION DIG-T S- - 1332cc,11:00 Wednesday 5th February 2025,https://www.ebca.co.uk/images/imagecomingsoon.jpg,0,GY70DYS,22 January 2021,NISSAN,QASHQAI,N-MOTION DIG-T S- - 1332cc,5,...,Automatic,Petrol,28000 Correct,0,https://ebca-media.kfsnet.co.uk/i_folder/GY70D...,Hatchback,NaN,NaN,NaN,NaN
